In [1]:
# Importing necessary libraries
import pandas as pd
import sqlite3

# Connect to the SQLite database
database_path = 'nba_salary.sqlite'  # Replace with your file path if needed
conn = sqlite3.connect(database_path)

# Function to preview data from a table
def preview_table(connection, table_name, limit=5):
    query = f"SELECT * FROM {table_name} LIMIT {limit};"
    return pd.read_sql_query(query, connection)

# Fetch and preview table names
def get_table_names(connection):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql_query(query, connection)

print("Tables in the database:")
print(get_table_names(conn))

# Preview data from the tables
nba_salary_preview = preview_table(conn, "NBA_season1718_salary")
season_stats_preview = preview_table(conn, "Seasons_Stats")

print("NBA Salary Table Preview:")
print(nba_salary_preview.head())

print("Season Stats Table Preview:")
print(season_stats_preview.head())

# Perform analysis: Join salary and performance data
query = """
SELECT 
    salary.Player,
    salary.Tm AS Team,
    salary.season17_18 AS Salary,
    stats.PER,
    stats.PTS,
    stats.WS,
    stats.Age,
    stats.Pos
FROM 
    NBA_season1718_salary AS salary
JOIN 
    Seasons_Stats AS stats
ON 
    salary.Player = stats.Player
WHERE 
    stats.Year = 2017
ORDER BY 
    Salary DESC;
"""
analysis_result = pd.read_sql_query(query, conn)

# Display the analysis results
print("Analysis: Salary vs. Performance")
print(analysis_result)

# Close the database connection
conn.close()


Tables in the database:
                    name
0  NBA_season1718_salary
1          Seasons_Stats
NBA Salary Table Preview:
    X1          Player   Tm  season17_18
0  1.0   Stephen Curry  GSW   34682550.0
1  2.0    LeBron James  CLE   33285709.0
2  3.0    Paul Millsap  DEN   31269231.0
3  4.0  Gordon Hayward  BOS   29727900.0
4  5.0   Blake Griffin  DET   29512900.0
Season Stats Table Preview:
    X1    Year           Player  Pos   Age   Tm     G    GS    MP   PER  ...  \
0  0.0  1950.0  Curly Armstrong  G-F  31.0  FTW  63.0  None  None  None  ...   
1  1.0  1950.0     Cliff Barker   SG  29.0  INO  49.0  None  None  None  ...   
2  2.0  1950.0    Leo Barnhorst   SF  25.0  CHS  67.0  None  None  None  ...   
3  3.0  1950.0       Ed Bartels    F  24.0  TOT  15.0  None  None  None  ...   
4  4.0  1950.0       Ed Bartels    F  24.0  DNN  13.0  None  None  None  ...   

     FT%   ORB   DRB   TRB    AST   STL   BLK   TOV     PF    PTS  
0  0.705  None  None  None  176.0  None  None  None 

In [ ]:
query = "SELECT * FROM Observation LIMIT 5;"
pd.read_sql(query, conn)